In [1]:
%matplotlib inline
import re
import collections
import string

In [2]:
test_data = """Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.""".splitlines()

In [3]:
with open('inputs/day7.txt') as fp:
    real_data = fp.read().splitlines()

### Part 1 ###

In [4]:
prereq_re = re.compile(r'Step (\w) must be finished before step (\w)')
m = prereq_re.match(test_data[0])
assert ('C', 'A') == m.groups()

In [5]:
def parse_prereq_lines(lines):
    nodes = set()
    prereqs = collections.defaultdict(list)
    for line in lines:
        requires, node = prereq_re.match(line).groups()
        nodes.add(requires)
        nodes.add(node)
        prereqs[node].append(requires)
    return prereqs, list(nodes)

In [6]:
test_prereqs, test_nodes = parse_prereq_lines(test_data)
test_nodes

['E', 'F', 'D', 'C', 'B', 'A']

In [7]:
real_prereqs, real_nodes = parse_prereq_lines(real_data)

In [8]:
def new_prereqs_without_node(prereqs, node):
    new_prereqs = {}
    for end_node in prereqs:
        lst = [p for p in prereqs[end_node] if p != node]
        if lst != []:
            new_prereqs[end_node] = lst
    return new_prereqs

In [9]:
new_prereqs_without_node(test_prereqs, 'C')

{'B': ['A'], 'D': ['A'], 'E': ['B', 'D', 'F']}

In [10]:
def process(nodes, prereqs):
    curr_nodes = nodes.copy()
    while curr_nodes:
        avail = sorted(set(curr_nodes) - set(prereqs.keys()))[0]
        yield avail
        curr_nodes.remove(avail)
        prereqs = new_prereqs_without_node(prereqs, avail)

In [11]:
assert 'CABDFE' == ''.join(process(test_nodes, test_prereqs))

In [12]:
''.join(process(real_nodes, real_prereqs))

'GRTAHKLQVYWXMUBCZPIJFEDNSO'

### Part 2 ###

In [13]:
def next_available_in_pool(pool):
    for w in pool:
        if w[0] is None:
            return w
    return None

def process_with_workers(nodes, prereqs, num_workers, base_time):
    timings = {c:i+base_time+1 for i,c in enumerate(string.ascii_uppercase)}
    curr_nodes = nodes.copy()
    t = 0
    pool = [[None, 0] for _ in range(num_workers)] # Each worker is a [node, finish-time] pair
    while curr_nodes:
        claimed = [w[0] for w in pool if w[0] is not None]
        avail = sorted(set(curr_nodes) - set(prereqs.keys()) - set(claimed))
        if avail:
            for node in avail:
                w = next_available_in_pool(pool)
                if w is not None:
                    w[0] = node
                    w[1] = t+timings[node]
        # available nodes assigned, skip to the time that the first finishes
        t = min(w[1] for w in pool if w[0] is not None)
        for w in pool:
            if w[1] == t:
                node = w[0]
                yield t, node
                w[0] = None
                curr_nodes.remove(node)
                prereqs = new_prereqs_without_node(prereqs, node)

In [14]:
test_worker_seq = list(process_with_workers(test_nodes, test_prereqs, 2, 0))
assert test_worker_seq[-1] == (15, 'E')

In [15]:
puzzle_worker_seq = list(process_with_workers(real_nodes, real_prereqs, 5, 60))

In [16]:
puzzle_worker_seq[-1]

(1115, 'O')